In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random

# from data.data_accessors import tmprss2_to_pandas
import numpy as np
from rdkit import Chem, DataStructs
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pickle

random.seed(1)

os.chdir('/Users/JacobUmans/tmprss2/notebooks')

In [3]:
# collect dataset
dataset = pd.read_pickle("../dumps/combined_dataset.pkl")
dataset

,acvalue_target,activity_target,target,tmprss2_vs_target_spearman_r,acvalue_scaled_to_tmprss2,cid,morgan_fingerprint
0,0.019,Active,TMPRSS2,1.000000,0.019,46899577,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0.019,Active,TMPRSS2,1.000000,0.019,56677005,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0.020,Active,TMPRSS2,1.000000,0.020,49835415,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0.021,Active,TMPRSS2,1.000000,0.021,49835557,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0.044,Active,TMPRSS2,1.000000,0.044,0,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
890,0.168,Active,TMPRSS11D,0.064941,NaN,49835606,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
891,0.038,Active,TMPRSS11D,0.064941,NaN,56680339,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
892,0.043,Active,TMPRSS11D,0.064941,NaN,56683657,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
893,0.183,Active,TMPRSS11D,0.064941,NaN,56683658,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [4]:
fingerprints = dataset.morgan_fingerprint
X = np.zeros([len(dataset), 2048])
for i in range(len(dataset)):
    for j in range(2048):
        X[i][j] = fingerprints[i][j]

y = np.array(dataset.acvalue_scaled_to_tmprss2)

X = X[~np.isnan(y)]

y = -np.log10(y)
y = y[~np.isnan(y)]

In [5]:
#split training and testing data for each dataset

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

In [6]:
#SVM regression
svmReg = make_pipeline(StandardScaler(), LinearSVR(random_state=0, tol=1e-5))
svmReg.fit(train_X, train_y)

pred = svmReg.predict(test_X)
MSE = mean_squared_error(test_y, pred)
RMSE = np.sqrt(MSE)
print("SVR RMSE:{}".format(RMSE))

SVR RMSE:1.0927675717987686


In [7]:
# collect dataset
broad_library = pd.read_csv('../data/broad_repurposing_library.csv')
broad_library = broad_library.drop(9603)
broad_library = broad_library.drop(9602)
broad_library = broad_library.drop(9040)
broad_library = broad_library.drop(8416)
broad_library = broad_library.drop(7224)
broad_library = broad_library.drop(2548)
broad_library['fingerprints'] = broad_library.smiles.apply(lambda s: Chem.RDKFingerprint(Chem.MolFromSmiles(s)))


In [8]:
broad_fingerprints = np.zeros([len(broad_library), 2048])
for i in range(len(broad_library)):
    DataStructs.ConvertToNumpyArray(broad_library.fingerprints.iloc[i], broad_fingerprints[i])

In [13]:
svmReg_full = make_pipeline(StandardScaler(), LinearSVR(random_state=0, tol=1e-5))
svmReg_full.fit(X, y)
predictions = np.zeros([len(broad_fingerprints), 2])
pred_values = svmReg_full.predict(broad_fingerprints)

In [14]:
argsort = np.argsort(pred_values)
compounds_sorted = broad_library.smiles[argsort]
pred_values_sorted = pred_values[argsort]


[-30.39500154 -30.39500154 -30.39500154 ...  13.06111162  13.06111162
  13.06111162]
8510              COc1ccc(CCCOC(Cn2ccnc2)c2ccc(OC)cc2)cc1
8508                                     CN(C)CCCCSC(N)=N
8509                                     CN(C)CCCCSC(N)=N
8992    CC(=O)Nc1cccc(Nc2ncnc(n2)N2CCC(CC2)OCc2ccc(OC(...
5419        Nc1nc(SCc2ncc[nH]2)c(C#N)c(-c2ccc(O)cc2)c1C#N
                              ...                        
279     CC(C)CCC[C@@H](C)CCC[C@@H](C)CCC[C@@](C)(O)CCC...
280     CC(C)CCC[C@@H](C)CCC[C@@H](C)CCC[C@@](C)(O)CCC...
2545    COC1=C(OC)C(=O)C(C\C=C(/C)CC\C=C(/C)CC\C=C(/C)...
2544    COC1=C(OC)C(=O)C(C\C=C(/C)CC\C=C(/C)CC\C=C(/C)...
2543    COC1=C(OC)C(=O)C(C\C=C(/C)CC\C=C(/C)CC\C=C(/C)...
Name: smiles, Length: 10141, dtype: object


In [17]:
Chem.MolFromSmiles(compounds_sorted[2543])
Chem.MolFromSmiles(compounds_sorted[2544])
Chem.MolFromSmiles(compounds_sorted[2545])

TypeError: No registered converter was able to produce a C++ rvalue of type std::__1::basic_string<wchar_t, std::__1::char_traits<wchar_t>, std::__1::allocator<wchar_t> > from this Python object of type numpy.float64